# Vehicle dynamics

The position of vehicle $i$ is $x_i(t)$ The distance to the vehicle or object
in front of vehicle $i$ is $q_i(t)$.

The vehicle speed $v_i(t) = \dot{x}_i(t)$. 

The control input to the vehicle is the acceleration $u_i$.

The desired distance to the obstacle in front $\hat{q}_i$ is calculated by scaling
with the current vehicle speed:

$$ \hat{q}_i(t) = k_q v_i(t) $$

Define a PD controller as:

$$ r_i(t) = q_i(t) - \hat{q}_i(t) $$
$$ u_i = k_p r_i(t) + k_d \frac{r_i(t) - r_i(t-\Delta t)}{\Delta t} $$


In [1]:
%matplotlib qt
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.transforms import Affine2D
from matplotlib.collections import CircleCollection
from matplotlib.widgets import CheckButtons


In [2]:
class RandomTraffic:
    def __init__(self, 
        n_vehicles: int,
        track_length: float,
        stop_traffic: bool,
        k_p: float, 
        k_d: float,
        k_q: float,
        k_qmin: float,
        v_min: float,
        v_max: float,
        u_min: float = -10.0,
        u_max: float = 1.0
    ) -> None:
        self.n_vehicles = n_vehicles
        self.track_length = track_length
        self.stop_traffic = stop_traffic
        self.k_p = k_p
        self.k_d = k_d
        self.k_q = k_q
        self.k_qmin = k_qmin
        self.v_min = v_min
        self.v_max = v_max
        self.u_min = u_min
        self.u_max = u_max
        self.position = np.zeros(n_vehicles, dtype=np.float32)
        self.velocity = np.zeros(n_vehicles, dtype=np.float32)
        self.active = np.zeros(n_vehicles, dtype=np.bool8)
        self.r_old = np.zeros(n_vehicles, dtype=np.float32)
    
    def enable_vehicle(self, init_speed: float = 0):
        """Find the first available vehicle slot and set active to True"""
        argmin = self.active.argmin()
        
        if not self.active[argmin]:
            # If argmin is already active, there's no space to add another vehicle
            self.active[argmin] = True
            self.position[argmin] = 0.0
            self.velocity[argmin] = init_speed

    def q(self):
        p = self.position[self.active]
        distance = np.zeros_like(p)

        order = np.argsort(p)
        p_in_front = np.concatenate([p[order][1:], [self.track_length*2]])
        if self.stop_traffic:
            stop_position = self.track_length * 0.5

            i = np.count_nonzero([p < stop_position])
            if i > 0:
                p_in_front[i-1] = stop_position

        diff = p_in_front - p[order]
        distance[order] = diff
        output = np.zeros_like(self.position)
        output[self.active] = distance        
        return output

    def qhat(self):
        return np.max([self.k_qmin * np.ones_like(self.velocity), self.k_q * self.velocity], axis=0)

    def r(self):
        return self.q() - self.qhat()

    def r_diff(self, dt):
        return (self.r() - self.r_old) / dt

    def u(self, dt):
        value = self.k_p * self.r() + self.k_d * self.r_diff(dt)
        return np.clip(value, self.u_min, self.u_max)
        
    def step(self, dt: float):
        u_calc = self.u(dt)
        v_calc = self.velocity + dt * u_calc
        v_calc = np.clip(v_calc, self.v_min, self.v_max)
        x_calc = self.position + dt * v_calc

        self.velocity = v_calc
        self.position = x_calc

        self.active = self.active * (self.position < self.track_length)
        self.position[self.active == False] = 0.0

        self.r_old = self.r()
        

In [3]:
class Circles:
    def __init__(self, traffic: RandomTraffic):
        self.traffic = traffic
        self.fig = plt.figure(figsize=(5, 1))
        self.ax = self.fig.add_axes([0, 0, 1, 1], frameon=False)
        self.ax.set_xlim([0, 200])
        self.ax.set_ylim([0, 20])

        self.circles = CircleCollection([], animated=True)
        self.ax.add_artist(self.circles)

        self.check_ax = self.fig.add_axes([0.7, -0.2, 0.2, 0.9], frameon=False)
        self.check_buttons = CheckButtons(self.check_ax, ['Stop traffic'], [traffic.stop_traffic])
        for label in self.check_buttons.labels:
            label.set_fontsize(10)

        def on_click(label):
            traffic.stop_traffic = not traffic.stop_traffic
        self.check_buttons.on_clicked(on_click)

    def update(self):
        positions = self.traffic.position[self.traffic.active]
        self.circles.set_sizes(np.ones_like(positions) * 10)
        self.circles.set_offsets(np.c_[positions, 50 * np.ones_like(positions)])
        return [self.circles, *[y for l in self.check_buttons.lines for y in l]]


In [5]:
t = RandomTraffic(
    n_vehicles=100,
    track_length=400,
    stop_traffic=False,
    k_p=100,
    k_d=10,
    k_q=1,
    k_qmin=8,
    v_min=0,
    v_max=30,
    u_max=10,
    u_min=-40
)
t.enable_vehicle(10)
circles = Circles(t)

all_positions = []
all_times = []

current_time = 0.0
def update(frame):
    global current_time, all_times, all_positions
    if np.random.rand() < 0.02:
        t.enable_vehicle(10)
    t.step(0.05)
    current_time += 0.05
    all_times.append(current_time)
    all_positions.append(t.position)
    return circles.update()

anim = FuncAnimation(circles.fig, update, interval=50, blit=True)


In [6]:
plt.plot(all_positions)

In [32]:
t.q()

[ 26.40999603 800.        ]
[26.40999603 80.        ]
[ 8.70882225 53.59000397]
[53.590004  8.708822]


array([53.590004,  8.708822,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      , 

In [15]:
position = np.array([4.0,3.5,2,1.4,0])
active = np.array([False, True, True, False, False])
output = np.zeros_like(position)
d = -np.diff(position[active])
output[active] = np.concatenate([[100], d])
output

array([  0. , 100. ,   1.5,   0. ,   0. ])

In [53]:
position = np.array([0.0, 0.0, 0.0, 12, 0.0, 3.4])
active = np.array([True, True, True, False, False, False])

p = position[active]
out = np.zeros_like(p)

order = np.argsort(p)
diff = np.diff(p[order])
out[order] = np.concatenate([diff, [100]])
output = np.zeros_like(position)
output[active] = out

In [54]:
output

array([  0.,   0., 100.,   0.,   0.,   0.])